# Certified Finetuning of a Classifier on the OCT-MNIST Dataset

In [3]:
%load_ext autoreload
%autoreload 2
import os
import torch
import tqdm
import abstract_gradient_training as agt
from abstract_gradient_training import AGTConfig
from abstract_gradient_training import certified_training_utils as ct_utils
from models.deepmind import DeepMindSmall 
from datasets import oct_mnist

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Pre-train the model

Exclude class 2 (Drusen) from the pretraining.

In [4]:
# set up pre-training
torch.manual_seed(1)
device = torch.device("cuda:0")
pretrain_batchsize = 100
pretrain_n_epochs = 20
pretrain_learning_rate = 0.001

In [5]:
# define model, dataset and optimizer
model = DeepMindSmall(1, 1)
dl_pretrain, _ = oct_mnist.get_dataloaders(pretrain_batchsize, exclude_classes=[2], balanced=True)
criterion = torch.nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=pretrain_learning_rate)
model = model.to(device)

In [6]:
if os.path.exists("medmnist.ckpt"):
    model.load_state_dict(torch.load("medmnist.ckpt"))
else:  # pre-train the model
    progress_bar = tqdm.trange(pretrain_n_epochs, desc="Epoch", )
    for epoch in progress_bar:
        for i, (x, u) in enumerate(dl_pretrain):
            # Forward pass
            u, x = u.to(device), x.to(device)
            output = model(x)
            loss = criterion(output.squeeze().float(), u.squeeze().float())
            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            if i % 100 == 0:
                progress_bar.set_postfix(loss=loss.item())
    # save the model
    with open("medmnist.ckpt", "wb") as file:
        torch.save(model.state_dict(), file)

## Fine-tune the model on the private Drusen data

In [7]:
# set up fine-tuning parameters
batchsize = 3000
config = AGTConfig(
    fragsize=1000,
    learning_rate=0.1,
    n_epochs=2,
    k_unlearn=50,
    forward_bound="interval",
    device="cuda:0",
    backward_bound="interval",
    loss="binary_cross_entropy",
    lr_decay=0.3,
    lr_min=0.001,
)

In [8]:
# get dataloaders, train dataloader is a mix of drusen and the "healthy" class
dl_train, _ = oct_mnist.get_dataloaders(batchsize, 1000, exclude_classes=[0, 1], balanced=True)
_, dl_test_drusen = oct_mnist.get_dataloaders(batchsize, 1000, exclude_classes=[0, 1, 3])
_, dl_test_other = oct_mnist.get_dataloaders(batchsize, 1000, exclude_classes=[2])
_, dl_test_all = oct_mnist.get_dataloaders(batchsize, 1000)

In [9]:
# evaluate the pre-trained model
param_n, param_l, param_u = ct_utils.get_parameters(model)
drusen_acc = agt.test_metrics.test_accuracy(param_n, param_l, param_u, *next(iter(dl_test_drusen)), model, ct_utils.propagate_conv_layers)
other_acc = agt.test_metrics.test_accuracy(param_n, param_l, param_u, *next(iter(dl_test_other)), model, ct_utils.propagate_conv_layers)
all_acc = agt.test_metrics.test_accuracy(param_n, param_l, param_u, *next(iter(dl_test_all)), model, ct_utils.propagate_conv_layers)

print("=========== Pre-trained model accuracy ===========")
print(f"Class 2 (Drusen) : nominal = {drusen_acc[1]:.2g}")
print(f"Classes 0, 1, 3  : nominal = {other_acc[1]:.2g}")
print(f"All Classes      : nominal = {all_acc[1]:.2g}")

=========== Pre-trained model accuracy ===========
Class 2 (Drusen) : nominal = 0.58
Classes 0, 1, 3  : nominal = 0.95
All Classes      : nominal = 0.86


In [10]:
# fine-tune the model using abstract gradient training (keeping the convolutional layers fixed)
param_l, param_n, param_u = agt.unlearning_certified_training(
    model, config, dl_train, dl_test_drusen, transform=ct_utils.propagate_conv_layers
)

[AGT] [INFO    ] [17:08:25] =================== Starting Unlearning Certified Training ===================
[AGT] [INFO    ] [17:08:25] Starting epoch 1
[AGT] [INFO    ] [17:08:25] Training batch 0: Network eval bounds=(0.58, 0.58, 0.58), W0 Bound=0.0 
[AGT] [INFO    ] [17:08:26] Training batch 1: Network eval bounds=(0.69, 0.75, 0.76), W0 Bound=0.0784 
[AGT] [INFO    ] [17:08:26] Training batch 2: Network eval bounds=(0.76, 0.82, 0.86), W0 Bound=0.153 
[AGT] [INFO    ] [17:08:27] Training batch 3: Network eval bounds=(0.77, 0.86, 0.89), W0 Bound=0.23 
[AGT] [INFO    ] [17:08:27] Training batch 4: Network eval bounds=(0.74, 0.86, 0.93), W0 Bound=0.313 
[AGT] [INFO    ] [17:08:28] Starting epoch 2
[AGT] [INFO    ] [17:08:28] Training batch 5: Network eval bounds=(0.69, 0.88, 0.94), W0 Bound=0.401 
[AGT] [INFO    ] [17:08:28] Training batch 6: Network eval bounds=(0.65, 0.88, 0.94), W0 Bound=0.496 
[AGT] [INFO    ] [17:08:29] Training batch 7: Network eval bounds=(0.6 , 0.88, 0.96), W0 Bo

In [11]:
# evaluate the fine-tuned model
drusen_acc = agt.test_metrics.test_accuracy(param_n, param_l, param_u, *next(iter(dl_test_drusen)), model, ct_utils.propagate_conv_layers)
other_acc = agt.test_metrics.test_accuracy(param_n, param_l, param_u, *next(iter(dl_test_other)), model, ct_utils.propagate_conv_layers)
all_acc = agt.test_metrics.test_accuracy(param_n, param_l, param_u, *next(iter(dl_test_all)), model, ct_utils.propagate_conv_layers)

print("=========== Fine-tuned model accuracy + bounds ===========")
print(f"Class 2 (Drusen) : nominal = {drusen_acc[1]:.2g}, certified bound = {drusen_acc[0]:.2g}")
print(f"Classes 0, 1, 3  : nominal = {other_acc[1]:.2g}, certified bound = {other_acc[0]:.2g}")
print(f"All Classes      : nominal = {all_acc[1]:.2g}, certified bound = {all_acc[0]:.2g}")

=========== Fine-tuned model accuracy + bounds ===========
Class 2 (Drusen) : nominal = 0.89, certified bound = 0.43
Classes 0, 1, 3  : nominal = 0.88, certified bound = 0.69
All Classes      : nominal = 0.88, certified bound = 0.63
